In [120]:
# Importojm librarit e nevojshme
import pandas as pd
import numpy as np

In [121]:
# Marrim csv file dhe e kthejm ne nje DataFrame
df = pd.read_csv("dirty_videogames_sales.csv")

print(df)

        Rank                                              Name Platform  \
0          1                                        Wii Sports      Wii   
1          2                                 Super Mario Bros.      NES   
2          3                                    Mario Kart Wii      Wii   
3          4                                 Wii Sports Resort      Wii   
4          5                          Pokemon Red/Pokemon Blue       GB   
...      ...                                               ...      ...   
16593  16596                Woody Woodpecker in Crazy Castle 5      GBA   
16594  16597                     Men in Black II: Alien Escape       GC   
16595  16598  SCORE International Baja 1000: The Official Game      PS2   
16596  16599                                        Know How 2       DS   
16597  16600                                  Spirits & Spells      GBA   

         Year         Genre   Publisher  NA_Sales  EU_Sales  JP_Sales  \
0      2006.0        Sport

In [122]:
# Krijojm nje kopje te datasetit qe te mos e ndryshojm origjinalin
df_clean = df.copy() 

In [123]:
df.columns

Index(['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'],
      dtype='object')

In [124]:
# Standardizojm emrat e kolonave (lowercase, spaced by underline, no spaces around the word)
df_clean.columns = df_clean.columns.str.strip().str.replace(" ", "_").str.lower()
df_clean.columns

# Standardizojm Publisher dhe Genre në formatin Title Case
df_clean['publisher'] = df_clean['publisher'].str.title()
df_clean['genre'] = df_clean['genre'].str.title()


In [125]:
# Shikojm per type te kolonave
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   rank          16598 non-null  int64  
 1   name          16598 non-null  object 
 2   platform      16598 non-null  object 
 3   year          16327 non-null  float64
 4   genre         16598 non-null  object 
 5   publisher     16540 non-null  object 
 6   na_sales      16598 non-null  float64
 7   eu_sales      16598 non-null  float64
 8   jp_sales      16598 non-null  float64
 9   other_sales   16598 non-null  float64
 10  global_sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


In [126]:
# Shikojm per missing values(NaN)
df_clean.isna().sum()

rank              0
name              0
platform          0
year            271
genre             0
publisher        58
na_sales          0
eu_sales          0
jp_sales          0
other_sales       0
global_sales      0
dtype: int64

In [127]:
# Kolonat me pa vlera(NaN) i vendosim ne nje list 
cols_to_clean = ["year","publisher"]

In [128]:
# Shohim se a ka vlera te barabarta dhe i largojm ato
df_clean.duplicated().sum()
df_clean.drop_duplicates(inplace=True)

In [129]:
# Largojm(drop) rreshtat qe nuk kan vlera tek kolonat e caktuara me lart
df_clean = df_clean.dropna(subset=cols_to_clean)

# Konvertoni 'year' në numër të plotë (int)
df_clean['year'] = df_clean['year'].astype(int)

In [130]:
df_clean

,rank,name,platform,year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010,Puzzle,7G//Ames,0.00,0.01,0.00,0.00,0.01


In [131]:
# Shohim se a eshte bere kalkulimi te global_sales ne menyr te duhur

region_cols= ["na_sales","eu_sales","jp_sales","other_sales"]
region_sum = df_clean[region_cols].sum(axis=1) 
df_clean['discrepancy'] = df_clean['global_sales'] - region_sum


In [132]:
# Rreshtat me ndryshim > 0.1
df_clean['review_flag'] = df_clean['discrepancy'].abs() > 0.1

# Rreshtat me ndryshim <= 0.01 (rrumbullakosje e vogël)
df_clean['rounding_issue'] = df_clean['discrepancy'].abs() <= 0.01

In [133]:
df_clean.loc[df_clean['review_flag'], region_cols] = np.nan


In [134]:
df_clean.isna().sum()

rank              0
name              0
platform          0
year              0
genre             0
publisher         0
na_sales          0
eu_sales          0
jp_sales          0
other_sales       0
global_sales      0
discrepancy       0
review_flag       0
rounding_issue    0
dtype: int64

In [135]:
# Mbushim rreshtat NaN proporcionalisht nga global_sales
region_cols= ["na_sales","eu_sales","jp_sales","other_sales"]
df_clean['known_sum'] = df_clean[region_cols].sum(axis=1, skipna=True)

for col in region_cols:
    df_clean[col + '_ratio'] = df_clean[col] / df_clean['known_sum']

for col in region_cols:
    ratio_col = col + '_ratio'
    # Vetëm rreshtat ku kolona është NaN
    mask = df_clean[col].isna() & df_clean[ratio_col].notna()
    df_clean.loc[mask, col] = df_clean.loc[mask, ratio_col] * df_clean.loc[mask, 'global_sales']

In [136]:
df_clean.drop(columns=[c+'_ratio' for c in region_cols] + ['known_sum'], inplace=True)


In [137]:
region_cols = ["na_sales", "eu_sales", "jp_sales", "other_sales"]

# 1. Llogarit shumen e rajoneve të njohura
df_clean['known_sum'] = df_clean[region_cols].sum(axis=1, skipna=True)

# 2. Flag për rreshtat ku të gjitha rajonet janë NaN
all_nan_mask = df_clean['known_sum'] == 0
df_clean['all_regions_missing'] = all_nan_mask

# 3. Proporcionet për rreshtat me disa rajone të njohura
for col in region_cols:
    df_clean[col + '_ratio'] = df_clean[col] / df_clean['known_sum']

# 4. Rimbush proporcionalisht
for col in region_cols:
    ratio_col = col + '_ratio'
    mask = df_clean[col].isna() & df_clean[ratio_col].notna()
    df_clean.loc[mask, col] = df_clean.loc[mask, ratio_col] * df_clean.loc[mask, 'global_sales']

# 5. Rimbush barabartë për rreshtat me të gjitha rajonet NaN
df_clean.loc[all_nan_mask, 'na_sales'] = df_clean.loc[all_nan_mask, 'global_sales'] / 4
df_clean.loc[all_nan_mask, 'eu_sales'] = df_clean.loc[all_nan_mask, 'global_sales'] / 4
df_clean.loc[all_nan_mask, 'jp_sales'] = df_clean.loc[all_nan_mask, 'global_sales'] / 4
df_clean.loc[all_nan_mask, 'other_sales'] = df_clean.loc[all_nan_mask, 'global_sales'] / 4

# 6. Fshij kolonat ndihmëse
df_clean.drop(columns=[c+'_ratio' for c in region_cols] + ['known_sum'], inplace=True)

# 7. Kontroll final
print("Numri i NaN në kolonat rajonale pas rimbushjes:")
print(df_clean[region_cols].isna().sum())


Numri i NaN në kolonat rajonale pas rimbushjes:
na_sales       0
eu_sales       0
jp_sales       0
other_sales    0
dtype: int64


In [138]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16291 entries, 0 to 16597
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   rank                 16291 non-null  int64  
 1   name                 16291 non-null  object 
 2   platform             16291 non-null  object 
 3   year                 16291 non-null  int64  
 4   genre                16291 non-null  object 
 5   publisher            16291 non-null  object 
 6   na_sales             16291 non-null  float64
 7   eu_sales             16291 non-null  float64
 8   jp_sales             16291 non-null  float64
 9   other_sales          16291 non-null  float64
 10  global_sales         16291 non-null  float64
 11  discrepancy          16291 non-null  float64
 12  review_flag          16291 non-null  bool   
 13  rounding_issue       16291 non-null  bool   
 14  all_regions_missing  16291 non-null  bool   
dtypes: bool(3), float64(6), int64(2), object(

In [139]:
cols_to_keep = [
    'name', 'platform', 'year', 'genre', 'publisher', 'rank',
    'na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'global_sales'
]

df_clean = df_clean[cols_to_keep]


In [140]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16291 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          16291 non-null  object 
 1   platform      16291 non-null  object 
 2   year          16291 non-null  int64  
 3   genre         16291 non-null  object 
 4   publisher     16291 non-null  object 
 5   rank          16291 non-null  int64  
 6   na_sales      16291 non-null  float64
 7   eu_sales      16291 non-null  float64
 8   jp_sales      16291 non-null  float64
 9   other_sales   16291 non-null  float64
 10  global_sales  16291 non-null  float64
dtypes: float64(5), int64(2), object(4)
memory usage: 1.5+ MB


In [141]:
# Krijojm nje kolone te re global_sales e cila eshte shuma e rajonaleve
df_clean['global_sales_recalculated'] = df_clean[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)

# Kontrollojm per ndryshime. Nese jeni te sigurte ne imputimin tuaj, mund ta zevendesoni.
df_clean['global_sales'] = df_clean['global_sales_recalculated']
df_clean.drop(columns=['global_sales_recalculated'], inplace=True)

C:\Users\genti\AppData\Local\Temp\ipykernel_6248\1270772285.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['global_sales_recalculated'] = df_clean[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)
C:\Users\genti\AppData\Local\Temp\ipykernel_6248\1270772285.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['global_sales'] = df_clean['global_sales_recalculated']
C:\Users\genti\AppData\Local\Temp\ipykernel_6248\1270772285.py:6: SettingWithCopyWarning: 
A value is trying 

In [142]:
df_clean

,name,platform,year,genre,publisher,rank,na_sales,eu_sales,jp_sales,other_sales,global_sales
0,Wii Sports,Wii,2006,Sports,Nintendo,1,41.4900,29.0200,3.7700,8.4600,82.74
1,Super Mario Bros.,NES,1985,Platform,Nintendo,2,29.0800,3.5800,6.8100,0.7700,40.24
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,3,15.8500,12.8800,3.7900,3.3100,35.83
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,4,15.7500,11.0100,3.2800,2.9600,33.00
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,5,11.2700,8.8900,10.2200,1.0000,31.38
...,...,...,...,...,...,...,...,...,...,...,...
16593,Woody Woodpecker in Crazy Castle 5,GBA,2002,Platform,Kemco,16596,0.0100,0.0000,0.0000,0.0000,0.01
16594,Men in Black II: Alien Escape,GC,2003,Shooter,Infogrames,16597,0.0100,0.0000,0.0000,0.0000,0.01
16595,SCORE International Baja 1000: The Official Game,PS2,2008,Racing,Activision,16598,0.0025,0.0025,0.0025,0.0025,0.01
16596,Know How 2,DS,2010,Puzzle,7G//Ames,16599,0.0000,0.0100,0.0000,0.0000,0.01


In [143]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16291 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          16291 non-null  object 
 1   platform      16291 non-null  object 
 2   year          16291 non-null  int64  
 3   genre         16291 non-null  object 
 4   publisher     16291 non-null  object 
 5   rank          16291 non-null  int64  
 6   na_sales      16291 non-null  float64
 7   eu_sales      16291 non-null  float64
 8   jp_sales      16291 non-null  float64
 9   other_sales   16291 non-null  float64
 10  global_sales  16291 non-null  float64
dtypes: float64(5), int64(2), object(4)
memory usage: 1.5+ MB


In [144]:
df_clean.isna().sum()

name            0
platform        0
year            0
genre           0
publisher       0
rank            0
na_sales        0
eu_sales        0
jp_sales        0
other_sales     0
global_sales    0
dtype: int64

In [145]:
# Kolonat e shitjeve per t'u rrumbullakosur
sales_cols = ["na_sales", "eu_sales", "jp_sales", "other_sales", "global_sales"]

# Rrumbullakosim te gjitha kolonat e shitjeve ne 2 shifra pas presjes
df_clean[sales_cols] = df_clean[sales_cols].round(2)

C:\Users\genti\AppData\Local\Temp\ipykernel_6248\2612406203.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[sales_cols] = df_clean[sales_cols].round(2)


In [146]:
df_clean.to_csv("cleaned_videogames_sales.csv", index=False)